# TwiScope9000




## PRERequisites: 

### Python 3
### Tweepy [pip install tweepy]
### Pandas [pip install pandas]
### Python Google search library [pip install google]
### Twitter API keys



In [17]:
# Module imports 
import tweepy
import time
import pandas as pd
import numpy as np
import json
import time
from random import randint, random
from googlesearch import search as gsearch
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

## Authentication

In [18]:
api_key = '5upK5p9UCIXPnZYGA0ihqiH4X'
secret_key = 'wx7WBGRK8n49YXC797tun2jFZGKL01FLuFI7WZ5JEZssFyupWK'
access_token = '1095267385496367105-fJkTQUk4HorYP5R6o3aXjeZL3Epto7'
secret_token = 'jpbfQbQaIOnnXaY8FvTa4NzxKIMPDLKz7m7w0pkidP9pk'

def authenticate(api_key, secret_key, access_token, secret_token):

    auth = tweepy.OAuthHandler(api_key, secret_key)
    auth.set_access_token(access_token, secret_token)
    api = tweepy.API(auth, 
                     wait_on_rate_limit=True, 
                     wait_on_rate_limit_notify=True)
  
    return api


api = authenticate(api_key, secret_key, access_token, secret_token)

increase sleep time for searching more lists. 

In [19]:
def limit_handled(cursor):
    
    while True:
        try:
            yield cursor.next()
        except tweepy.RateLimitError:
            time.sleep(2)
            continue

## Scrape lists that contain keyword_string

In [41]:

def scrape_lists_with_google(keyword_string, results_to_obtain):

    list_urls = []

    urls_checked = 0
    urls_appended = 0
    
    # Perform the Google search, checking only the twitter.com domain
    for url in gsearch("site:twitter.com lists " + keyword_string, 
                      start=urls_checked, 
                      num=10, 
                      pause=randint(3, 5)):
        
        # Take URLs that contain Twitter's '/lists' subdomain
        if '/lists/' in url:
            list_urls.append(url)
            urls_appended += 1
            urls_checked += 1

        # If Google is returning 5x as many non-list results as lists, break search
        if urls_checked / urls_appended > 5:
            break
        
        # If we reach our results # request limit, break the search
        elif urls_appended >= results_to_obtain:
            break
        
        # Sleep to prevent timeouts
        time.sleep(randint(3, 10))

    list_urls = [url[8:] for url in list_urls] # Removes leading https://

    return list_urls
  

list_urls = scrape_lists_with_google("WooCommerce", 10)

## Creating a dataframe with user data


In [42]:
# Get user id, screen name, bio, follower count

def get_users_in_lists(list_urls):
  
    users_list = []
    bios_list = []
    desc_list = []
    follower_count_list = []
    
    for tw_list in list_urls:
    
        user = tw_list.split('/')[1]
        list_name = tw_list.split('/')[3]

        # Using Tweepy api, getting json of all users in list
        list_members_output = api.list_members(user, list_name)

        # Parsing json data and appending to appropriate lists instantiated above
        for user_id in list_members_output:
            users_list.append(user_id._json['id'])
            bios_list.append(user_id._json['screen_name'])
            desc_list.append(user_id._json['description'])
            follower_count_list.append(user_id._json['followers_count'])
            
    zipped_data = zip(users_list, bios_list, desc_list, follower_count_list)
    
    # Creates a pandas dataframe containing user attributes
    output_df = pd.DataFrame(list(zipped_data), columns=['user_id',
                                                         'screen_name', 
                                                         'bio', 
                                                         'followers'])
    # Dropping duplicate users if they were in multiple lists
    output_df.drop_duplicates('user_id')
    
    output_df.set_index('user_id', inplace=True)
    
    return output_df
  

influencers_df = get_users_in_lists(list_urls)

In [44]:
influencers_df.sort_values(by='followers', ascending=False).head()
influencers_df.to_excel('influencersplain.xlsx')

## Remove protected accounts


In [45]:
def remove_protected_accounts(users_df):
    
    working_df = users_df.copy()
    protected_ids = []
    
    counter = 0
    for index, row in users_df.iterrows():
        user = api.get_user(id=index)._json
        if user['protected'] == True:
            protected_ids.append(index)
        counter += 1
        if counter % 100 == 0:
            print(str(counter) + '/' + str(len(working_df)) + ' IDs checked')
            print('Protected IDs found:', len(protected_ids), '\n')
        # Sleeping for up to 0.2s helps not to exceed Tweepy's rate limit.
        time.sleep(random() / 5)
    
    working_df.drop(labels=protected_ids, inplace=True)
    working_df.reset_index(inplace=True)
            
    return working_df
            

influencers_df = remove_protected_accounts(influencers_df)

100/141 IDs checked
Protected IDs found: 1 



## RT and Favorite calculation for each user

In [46]:
def calc_median_favorites(user_id):
    
    fav_list = []
    tweets = api.user_timeline(id=user_id, count=100)
    
    for tweet in tweets:
        # Remove retweets
        if tweet._json['text'].startswith('RT'):
            continue
        # Remove replies
        elif tweet._json['text'].startswith('@'):
            continue
        else:
            fav_list.append(tweet._json['favorite_count'])
    
    time.sleep(random() / 5)
    
    return np.median(fav_list)  


influencers_df['median_favs'] = influencers_df['user_id'].apply(lambda x: calc_median_favorites(x))

/Users/dheerajakshay/anaconda3/lib/python3.6/site-packages/numpy/core/fromnumeric.py:3118: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)


In [47]:
def calc_median_retweets(user_id):
    
    rt_list = []
    tweets = api.user_timeline(id=user_id, count=100)
    
    for tweet in tweets:
        if tweet._json['text'].startswith('RT'):
            continue
        else:
            rt_list.append(tweet._json['retweet_count'])
    
    time.sleep(random() / 5)
    
    return np.median(rt_list)  


influencers_df['median_rts'] = influencers_df['user_id'].apply(lambda x: calc_median_retweets(x))

In [49]:
influencers_df
influencers_df.to_excel('influencersengagement.xlsx')

## Refinement and Sorting based on Engagement 

In [34]:
def remove_low_engagement_accounts(df):
    
    working_df = df.copy()
    
    working_df = working_df.loc[(working_df['median_favs'] > 0) &
                                (working_df['followers'] >= 1000)]
    
    return working_df

influencers_df = remove_low_engagement_accounts(influencers_df)

In [35]:
def create_engagement_metric(df):
      
    working_df = df.copy()
    
    from sklearn.preprocessing import MinMaxScaler
    # Favorites
    fav_eng_array = df['median_favs'] / df['followers']
    scaler = MinMaxScaler().fit(fav_eng_array.values.reshape(-1, 1))
    scaled_favs = scaler.transform(fav_eng_array.values.reshape(-1, 1))
    
    # Retweets
    rt_eng_array = df['median_favs'] / df['followers']
    scaler = MinMaxScaler().fit(rt_eng_array.values.reshape(-1, 1))
    scaled_rts = scaler.transform(rt_eng_array.values.reshape(-1, 1))
    
    mean_eng = (scaled_favs + scaled_rts) / 2
    working_df['engagement'] = mean_eng
    
    return working_df
                                                

influencers_df = create_engagement_metric(influencers_df)

In [36]:
influencers_df.sort_values('engagement', ascending=False).head()

,user_id,screen_name,bio,followers,median_favs,median_rts,engagement
83,871118484478078976,wpgraphql,Open Source WordPress plugin that brings the p...,1358,20.0,0.0,1.000000
117,416488809,TopHatArsen,"Digital Marketer, SEO, International Speaker. ...",2509,8.0,0.0,0.213708
63,2341017673,AutomatticEng,The official account for @Automattic Engineeri...,2683,7.0,0.0,0.174220
20,294079511,pippinsplugins,"PHP developer, craft beer lover and brewer, av...",18681,43.5,0.0,0.155109
79,294079511,pippinsplugins,"PHP developer, craft beer lover and brewer, av...",18681,43.5,0.0,0.155109


In [37]:
influencers_df


,user_id,screen_name,bio,followers,median_favs,median_rts,engagement
0,1092837286230798336,presstitanwp,White Glove Web Services for Businesses - Focu...,1266,1.0,0.0,0.050260
18,3693311536,wpcampusorg,A community and conferences for #WordPress in ...,1966,2.0,2.0,0.065756
19,3428226791,WooConf,WordPress developers & eCommerce experts inter...,1975,1.0,0.0,0.030938
20,294079511,pippinsplugins,"PHP developer, craft beer lover and brewer, av...",18681,43.5,0.0,0.155109
21,128310837,chriscct7,Managing Partner of the Analytics and eCommerc...,1515,1.0,0.0,0.041414
22,99167392,jthomasgriffin,Co-Founder and President of @OptinMonster (htt...,4016,2.0,0.0,0.030371
23,85808180,tommcfarlin,@WebDevStudios 🖥 / @Pressware 💻 / Blogging ⌨️ ...,13999,1.0,0.0,0.001303
24,40516848,wpbeginner,WPBeginner is the largest unofficial #WordPres...,191179,10.0,0.0,0.000000
25,34998505,sucurisecurity,We clean and protect websites. Website Securit...,21971,4.0,1.0,0.008842
26,33521560,obenland,"Chairman of Height at @Automattic, WordPress C...",4259,2.5,0.0,0.036435


## TODO: .py files 